In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
from pkg.controller.robot_config import RobotType
from pkg.utils.test_scripts import *

ROBOT_TYPE = RobotType.indy7gripper
VISUALIZE = True

ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, HOME_POSE, GRIP_DEPTH = get_single_robot_params(ROBOT_TYPE)
s_builder, pscene = prepare_single_robot_scene(ROBOT_TYPE, ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, VISUALIZE=VISUALIZE)
crob, gscene = pscene.combined_robot, pscene.gscene

from pkg.planning.filtering.latticized_filter import SCENE_PATH, ARM_SHAPE, GRASP_SHAPE, \
    WDH_GRASP, L_CELL_GRASP, OFFSET_ZERO_GRASP, WDH_ARM, L_CELL_ARM, OFFSET_ZERO_ARM
from pkg.planning.filtering.lattice_model.latticizer_py import get_centers

centers_grasp = get_centers(GRASP_SHAPE, L_CELL_GRASP, OFFSET_ZERO_GRASP)
centers_arm = get_centers(ARM_SHAPE, L_CELL_ARM, OFFSET_ZERO_ARM)


assert len(pscene.robot_chain_dict.values())==1, "not single robot scene"
shoulder_height = get_tf(
    gscene.urdf_content.joint_map[pscene.robot_chain_dict.values()[0]['joint_names'][1]].child, 
    crob.home_dict, pscene.gscene.urdf_content)[2, 3]

connection command:
indy0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


In [2]:
file_idx = 612
scene_data = load_pickle(os.path.join(SCENE_PATH, "{:08d}-OK.pkl".format(file_idx)))
global_log = scene_data["global_log"]
arm_tar_idx = scene_data["arm_tar_idx"]
grasp_tool_idx = scene_data["grasp_tool_idx"]
grasp_tar_idx = scene_data["grasp_tar_idx"]
grasp_obj_idx = scene_data["grasp_obj_idx"]
radius, theta, height = scene_data["rth"]
gtem_args = scene_data["gtem_args"]
error_state = scene_data["error_state"]
offset_ee = cyl2cart(radius, theta, height)

while "obj_0" not in [gtem["name"] for gtem in gtem_args]:
    scene_data_ref = load_pickle(os.path.join(SCENE_PATH, "{:08d}-OK.pkl".format(file_idx-1)))
    gtem_args = scene_data_ref["gtem_args"]
    
load_gtem_args(gscene, gtem_args)

arm_tar_idx =  np.unravel_index(arm_tar_idx, shape=GRASP_SHAPE)
grasp_tool_idx = np.unravel_index(grasp_tool_idx, shape=GRASP_SHAPE)
grasp_tar_idx = np.unravel_index(grasp_tar_idx, shape=GRASP_SHAPE)
grasp_obj_idx = np.unravel_index(np.array(grasp_obj_idx).astype(np.int), shape=GRASP_SHAPE)
radius, theta, height = scene_data["rth"]

for i_c, center in enumerate(centers_arm[arm_tar_idx]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + [0,0,shoulder_height]
    gscene.create_safe(GEOTYPE.SPHERE, "arm_%05d"%(i_c), "base_link", dims=(0.03,) * 3,
                                            center=center_rot, rpy=(0, 0, 0), color=(0.5, 0.5, 0.5, 0.7),
                                            display=True, collision=False, fixed=True)
    
for i_c, center in enumerate(centers_grasp[grasp_tool_idx]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + offset_ee
    gscene.create_safe(GEOTYPE.SPHERE, "tool_%05d"%(i_c), "base_link", dims=(0.02,) * 3,
                                            center=center_rot, rpy=(0, 0, 0), color=(0, 1, 0, 0.7),
                                            display=True, collision=False, fixed=True)

for i_c, center in enumerate(centers_grasp[grasp_tar_idx]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + offset_ee
    gscene.create_safe(GEOTYPE.SPHERE, "tar_%05d"%(i_c), "base_link", dims=(0.02,) * 3,
                                            center=center_rot, rpy=(0, 0, 0), color=(1, 0, 0, 0.7),
                                            display=True, collision=False, fixed=True)
    
for i_c, center in enumerate(centers_grasp[grasp_obj_idx]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + offset_ee
    gscene.create_safe(GEOTYPE.SPHERE, "obj_%05d"%(i_c), "base_link", dims=(0.02,) * 3,
                                            center=center_rot, rpy=(0, 0, 0), color=(1, 1, 0, 0.7),
                                            display=True, collision=False, fixed=True)
    
from_state = global_log["from_state"]
to_state = global_log["to_state"]
subject = global_log["subject"]
actor = global_log["actor"]
print("{} - {}".format(actor, subject))

gscene.show_pose(from_state.Q)
gscene.show_pose(to_state.Q)

gp - obj_0


In [5]:
gscene.show_pose(from_state.Q)

In [6]:
gscene.show_pose(to_state.Q)

### find idx that deals with object other than obj_0

In [4]:
for file_idx in range(1173):
    scene_data = load_pickle(os.path.join(SCENE_PATH, "{:08d}-OK.pkl".format(file_idx)))
    global_log = scene_data["global_log"]
#     if global_log["subject"] != "obj_0" and global_log["actor"] != "grip0":
    if global_log["actor"] != "grip0":
        print(file_idx)

1
3
5
7
9
11
13
15
17
19
21
24
26
27
28
30
32
34
36
38
40
42
44
46
48
51
53
55
57
59
61
63
65
67
69
71
73
75
77
79
81
83
85
87
89
94
96
98
100
102
104
106
108
110
112
114
116
119
121
123
125
127
131
133
135
137
139
141
143
145
147
149
151
153
156
158
161
163
165
167
169
171
173
175
177
179
181
183
185
187
189
192
194
196
198
207
208
210
212
214
216
218
220
222
225
271
290
296
306
343
349
361
381
415
416
417
418
419
420
421
486
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
561
566
586
589
591
596
612


KeyboardInterrupt: 